In [1]:
import json
import pymysql

In [2]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [3]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port'] 
)

# 패스워드 암호화
hashlib(보안해시와 메시지요약)이 모듈은 다양한 보안 해시(secure hash)와 메시지 요약(message digest) 알고리즘에 대한 공통 인터페이스를 구현합니다. RSA의 MD5 알고리즘(Internet RFC 1321에서 정의됩니다)뿐만 아니라 FIPS 보안 해시 알고리즘 SHA1, SHA224, SHA256, SHA384 및 SHA512(FIPS 180-2에 정의됩니다)가 포함됩니다. 《보안 해시》와 《메시지 다이제스트》라는 용어는 서로 바꿔 사용할 수 있습니다. 오래된 알고리즘들은 메시지 요약이라고 불립니다. 현대 용어는 보안 해시입니다

In [4]:
import hashlib
import base64

## - Secure Hash Algorithm(SHA-256)

In [5]:
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

## - Base64로 인코딩

In [6]:
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [8]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [9]:
len(hashed_pwd)

44

In [10]:
# 패스워드를 생성하는 함수
def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

# user table을 읽어서 pwd 열에 패스워드를 채움

In [11]:
sql = 'SELECT uid FROM users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('diy',),
 ('eskim',),
 ('gdhong',),
 ('jbpark',),
 ('vincenzo',),
 ('wjlee',))

In [12]:
sql = 'UPDATE users SET pwd=%s where uid=%s;'
cur = conn.cursor()
for row in results:
    uid = row[0]
    pwd = gen_pwd(uid+'12')
    cur.execute(sql, (pwd, uid))
conn.commit()

# - 로그인

## 1) uid ok, pwd od

In [13]:
uid, pwd = 'admin', 'admin12'
sql = "SELECT uid, pwd FROM users WHERE uid=%s;"
cur = conn.cursor()
cur.execute(sql, (uid, ))  # (uid, ) 튜플이니깐 ,  붙어서
result = cur.fetchone()
db_pwd = result[1]
db_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

In [14]:
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

## 2) uid ok, pwd bad

In [15]:
pwd = '1234'
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

## 3) uid bad

In [16]:
uid = 'fool'

In [17]:
sql = "SELECT uid, pwd FROM users WHERE uid=%s;"
cur = conn.cursor()
cur.execute(sql, (uid, ))  # (uid, ) 튜플이니깐 ,  붙어서

0

In [18]:
result = cur.fetchone()
result

## 1,2,3을 하나로 합치기

In [ ]:
uid, pwd = 'admin', 'admin12'

In [23]:
sql = "SELECT uid, pwd FROM users WHERE uid=%s and is_deleted=0;"
cur = conn.cursor()
cur.execute(sql, (uid, ))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else:
        print('incorrect password')
else:
    print('bad uid')

bad uid


In [25]:
cur.close()
conn.close()

Error: Already closed

In [31]:
x = ''
if x:
    print('true')
else:
    print('false')

false


In [30]:
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else:
        print('incorrect password')
except:
    print('bad uid')

InterfaceError: (0, '')